In [1]:
#Install some stuff we need

!pip install ping3
!pip install ollama
!pip install docker


  Using cached ping3-4.0.8-py3-none-any.whl.metadata (13 kB)
Using cached ping3-4.0.8-py3-none-any.whl (14 kB)
  Using cached ollama-0.3.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached ollama-0.3.3-py3-none-any.whl (10 kB)
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
Using cached docker-7.1.0-py3-none-any.whl (147 kB)


In [4]:
from ping3 import ping, verbose_ping
verbose_ping('ollamafarm-nginx-proxy', count=4)
verbose_ping('ollamafarm_gpu_worker01', count=4)
verbose_ping('ollamafarm_gpu_worker02', count=4)
verbose_ping('ollamafarm_cpu_worker01', count=4)
verbose_ping('ollamafarm_cpu_worker02', count=4)
#verbose_ping('OllamaDev3', count=4)
#verbose_ping('OllamaDev4', count=4)


ping 'ollamafarm-nginx-proxy' ... 0ms
ping 'ollamafarm-nginx-proxy' ... 0ms
ping 'ollamafarm-nginx-proxy' ... 0ms
ping 'ollamafarm-nginx-proxy' ... 0ms
ping 'ollamafarm_gpu_worker01' ... 0ms
ping 'ollamafarm_gpu_worker01' ... 0ms
ping 'ollamafarm_gpu_worker01' ... 0ms
ping 'ollamafarm_gpu_worker01' ... 0ms
ping 'ollamafarm_gpu_worker02' ... 0ms
ping 'ollamafarm_gpu_worker02' ... 0ms
ping 'ollamafarm_gpu_worker02' ... 0ms
ping 'ollamafarm_gpu_worker02' ... 0ms
ping 'ollamafarm_cpu_worker01' ... 0ms
ping 'ollamafarm_cpu_worker01' ... 0ms
ping 'ollamafarm_cpu_worker01' ... 0ms
ping 'ollamafarm_cpu_worker01' ... 0ms
ping 'ollamafarm_cpu_worker02' ... 0ms
ping 'ollamafarm_cpu_worker02' ... 0ms
ping 'ollamafarm_cpu_worker02' ... 0ms
ping 'ollamafarm_cpu_worker02' ... 0ms


In [2]:
#Enumerate Workers
import docker

def get_virtual_hosts():
    # Connect to Docker using the default Unix socket
    client = docker.DockerClient(base_url='unix://tmp/docker.sock')

    virtual_hosts = {}

    # List all running containers
    containers = client.containers.list()

    # Iterate through the containers
    for container in containers:
        try:
            # Get container environment variables
            env_vars = container.attrs['Config']['Env']

            # Find VIRTUAL_HOST and VIRTUAL_PORT in environment variables
            virtual_host = None
            virtual_port = None

            for env in env_vars:
                if env.startswith("VIRTUAL_HOST="):
                    virtual_host = env.split("=")[1]
                if env.startswith("VIRTUAL_PORT="):
                    virtual_port = env.split("=")[1]

            # If VIRTUAL_HOST is set, add it to the virtual_hosts dictionary
            if virtual_host:
                virtual_hosts[virtual_host] = virtual_port if virtual_port else '80'  # Default port 80 if VIRTUAL_PORT is not set
        except KeyError:
            # Handle cases where environment variables might not be found
            continue

    return virtual_hosts


#Enumerate hosts
hosts = get_virtual_hosts()
if hosts:
    print("Virtual hosts found:")
    for host, port in hosts.items():
        print(f"Host: {host}, Port: {port}")
else:
    print("No virtual hosts found.")


DockerException: Error while fetching server API version: ('Connection aborted.', PermissionError(13, 'Permission denied'))

In [6]:
#Provision environments
import ollama
from ollama import Client

client = Client(host='http://ollamafarm_gpu_worker01:11434')
client.pull("llama3.1")

client = Client(host='http://ollamafarm_gpu_worker02:11434')
client.pull("llama3.1")






{'status': 'success'}

In [5]:
#Streaming test with the Load Balancer!
import ollama
from ollama import Client

client = Client(host='http://nginx-proxy:8080')
#client.pull("llama3.1")

stream = client.chat(
    model='llama3.1',
    #messages=[{'role': 'user', 'content': 'Why cant fish fly? One line only as a joke'}],
    messages=[{'role': 'user', 'content': 'Write me a song abour marmite'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

Here's a song about Marmite:

**Verse 1**
In the land of yeast and spices fine
There's a spread that's loved, yet hated in its prime
It's dark and it's thick, with a flavor so bold
Marmite, oh Marmite, you're a love to be told

**Chorus**
You're the blackest of spreads, but we adore
The way you make our toast, and sometimes more
You're the taste of umami, in every single bite
Marmite, oh Marmite, we can't live without your delight

**Verse 2**
Some say you're too strong, that your flavor's too bright
But for us, you're a treasure, a culinary sight
On crackers or on bread, you're the perfect pair
A sprinkle of cheese, and you'll show us you care

**Chorus**
You're the blackest of spreads, but we adore
The way you make our toast, and sometimes more
You're the taste of umami, in every single bite
Marmite, oh Marmite, we can't live without your delight

**Bridge**
In the UK, you're a national pride
A symbol of tradition, that won't be denied
So here's to you, dear Marmite, our love for you